In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
# Load in relevant libraries, and alias where appropriate
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import confusion_matrix
import numpy as np
from tensorflow.keras.datasets import mnist
import random

# Define relevant variables for the ML task
batch = 64
classes_num = 10
lrn_rt = 0.001
num_epochs = 10

# Device will determine whether to run the training on GPU or CPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
#Loading the dataset and preprocessing
train_dataset = torchvision.datasets.MNIST(root = './data',
                                           train = True,
                                           transform = transforms.Compose([
                                                  transforms.Resize((32,32)),
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(mean = (0.1307,), std = (0.3081,))]),
                                           download = True)


test_dataset = torchvision.datasets.MNIST(root = './data',
                                          train = False,
                                          transform = transforms.Compose([
                                                  transforms.Resize((32,32)),
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(mean = (0.1325,), std = (0.3105,))]),
                                          download=True)


train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch,
                                           shuffle = True)


test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = batch,
                                           shuffle = True)

In [3]:
#Defining the convolutional neural network
class LeNet5(nn.Module):
    def __init__(self, classes_num):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0)
        self.bn1 = nn.BatchNorm2d(6)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0)
        self.bn2 = nn.BatchNorm2d(16)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.fc1 = nn.Linear(400, 120)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(120, 84)
        self.relu4 = nn.ReLU()
        self.fc3 = nn.Linear(84, classes_num)
        
    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)
        out = self.pool1(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu2(out)
        out = self.pool2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu3(out)
        out = self.fc2(out)
        out = self.relu4(out)
        out = self.fc3(out)
        return out

In [4]:
model = LeNet5(classes_num).to(device)

#Setting the loss function
cost = nn.CrossEntropyLoss()

#Setting the optimizer with the model parameters and learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=lrn_rt)

#this is defined to print how many steps are remaining when training
total_step = len(train_loader)

In [5]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        images = images.to(device)
        labels = labels.to(device)
        
        #Forward pass
        outputs = model(images)
        loss = cost(outputs, labels)
        	
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        		
        if (i+1) % 400 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
        		           .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/10], Step [400/938], Loss: 0.0403
Epoch [1/10], Step [800/938], Loss: 0.1967
Epoch [2/10], Step [400/938], Loss: 0.0878
Epoch [2/10], Step [800/938], Loss: 0.0597
Epoch [3/10], Step [400/938], Loss: 0.0181
Epoch [3/10], Step [800/938], Loss: 0.0206
Epoch [4/10], Step [400/938], Loss: 0.0137
Epoch [4/10], Step [800/938], Loss: 0.0030
Epoch [5/10], Step [400/938], Loss: 0.0162
Epoch [5/10], Step [800/938], Loss: 0.0055
Epoch [6/10], Step [400/938], Loss: 0.0010
Epoch [6/10], Step [800/938], Loss: 0.0700
Epoch [7/10], Step [400/938], Loss: 0.0005
Epoch [7/10], Step [800/938], Loss: 0.0263
Epoch [8/10], Step [400/938], Loss: 0.0128
Epoch [8/10], Step [800/938], Loss: 0.0008
Epoch [9/10], Step [400/938], Loss: 0.0027
Epoch [9/10], Step [800/938], Loss: 0.1931
Epoch [10/10], Step [400/938], Loss: 0.0060
Epoch [10/10], Step [800/938], Loss: 0.0004


In [ ]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
  
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))
	 

In [ ]:


# Evaluate the model on the test set
model.eval()
y_true = []
y_pred = []
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        y_true += labels.cpu().numpy().tolist()
        y_pred += predicted.cpu().numpy().tolist()

# Print confusion matrix
cm = confusion_matrix(y_true, y_pred)
print('Confusion matrix:\n', cm)

# Print classification report
target_names = ['Class {}'.format(i) for i in range(classes_num)]
print('Classification report:\n', classification_report(y_true, y_pred, target_names=target_names))

In [ ]:

# get the predictions on the test set
with torch.no_grad():
    y_true = []
    y_pred = []
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        y_true += labels.cpu().numpy().tolist()
        y_pred += predicted.cpu().numpy().tolist()

# compute the confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# plot the confusion matrix
fig, ax = plt.subplots(figsize=(10, 10))
im = ax.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
ax.figure.colorbar(im, ax=ax)
classes = np.arange(10)
ax.set(xticks=np.arange(conf_matrix.shape[1]),
       yticks=np.arange(conf_matrix.shape[0]),
       xticklabels=classes, yticklabels=classes,
       xlabel='Predicted label',
       ylabel='True label',
       title='Confusion matrix')
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")
fmt = '.2f' 
thresh = conf_matrix.max() / 2.
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        ax.text(j, i, format(conf_matrix[i, j], fmt),
                ha="center", va="center",
                color="white" if conf_matrix[i, j] > thresh else "black")
fig.tight_layout()
plt.show()







In [ ]:


# initialize lists to store losses and epochs
train_losses = []
epochs = range(1, num_epochs+1)

# iterate through each epoch and collect the loss
for epoch in epochs:
    train_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = cost(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    # calculate and store the average loss for each epoch
    train_losses.append(train_loss / len(train_loader))

# plot the loss vs epochs graph
plt.plot(epochs, train_losses, label='Training Loss')
plt.title('Loss vs Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:


# Load MNIST dataset
(train_X, train_y), (test_X, test_y) = mnist.load_data()

# Count number of images in each class
class_counts = np.bincount(train_y)

# Plot bar graph of class distribution
plt.bar(range(10), class_counts)
plt.xticks(range(10), labels=[str(i) for i in range(10)])
plt.xlabel('Class')
plt.ylabel('Number of Images')
plt.title('MNIST Class Distribution')
plt.show()

In [ ]:
# Plot histogram of pixel intensity values
plt.hist(train_X.flatten(), bins=256, range=(0, 255))
plt.xlabel('Pixel Intensity')
plt.ylabel('Frequency')
plt.title('MNIST Pixel Intensity Distribution')
plt.show()

In [ ]:

# Plot random sample of 25 images
fig, ax = plt.subplots(5, 5, figsize=(8, 8))
ax = ax.flatten()
for i in range(25):
    img_index = random.randint(0, len(train_X))
    ax[i].imshow(train_X[img_index], cmap='gray')
    ax[i].set_title(f'Label: {train_y[img_index]}')
    ax[i].axis('off')
plt.show()